In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import Image, display
from sklearn.datasets import dump_svmlight_file
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from time import gmtime, strftime
import sys
import math
import json
import os 

In [0]:
# カルフォルニア大学アーバイン校の公開URLからデータセットをダウンロード
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o bank-additional.zip

In [0]:
# bank-additional-full.csvをdataへ格納
data = pd.read_csv('./bank-additional/bank-additional-full.csv', sep=';')
 
# 最初の10行を表示
data.head(10)

####	age：顧客の年齢
####	job：仕事のカテゴリ
####	marital：結婚ステータス
####	education：学歴
####	default：クレジットの支払遅延のステータス
####	housing：不動産ローンの有無
####	loan：パーソナルローンの有無
####	contact：銀行と顧客の連絡方法
####	month：最後に連絡をとった月
####	day_of_week：最後に連絡をとった曜日
####	duration：最後の連絡長さ（秒数）
####	campaign：このキャンペーン中に銀行が連絡した回数
####	pdays：最後のキャンペーンの連絡から経過した日にち
####	previous：このキャンペーン以前に銀行が連絡した回数
####	poutcome：前回のキャンペーンの結果
####	emp.var.rate：Employment Variation Rate
####	cons.price.idx：消費者物価指数
####	cons.conf.idx：消費者信頼感指数
####	euribor3m：3ヶ月スパンユーリボー指標金利
####	nr.employed：四半期の従業員数
####	y：顧客が定期預金を申し込みしたかどうか

In [0]:
# データフレームの欠損データをまとめるテーブルの関数
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns 

# dataに欠損があるかどうか確認
missing_values_table(data)

In [0]:
# 各項目の度数分布の表を作成
for column in data.select_dtypes(include=['object']).columns:
  display(pd.crosstab(index=data[column], 
                      columns='% observations', 
                      normalize='columns'))

In [0]:
# 数値を値として持っている項目のヒストグラム
display(data.describe())

hist = data.hist(bins=30, sharey=True, figsize=(10, 10))

In [0]:
# 各項目が予測ターゲット「y」にどのように関係してるか確認 
for column in data.select_dtypes(include=['object']).columns:
    if column != 'y':
        display(pd.crosstab(index=data[column], columns=data['y'], normalize='columns'))

for column in data.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = data[[column, 'y']].hist(by='y', bins=30)
    plt.show()

In [0]:
# 相関係数とScatter Matrixの確認
display(data.corr())
pd.plotting.scatter_matrix(data, figsize=(12, 12))
plt.show()

In [0]:
# 項目の追加
#data['add_columns'] = 


In [0]:
# カテゴリカルデータをダミー変数化
model_data = pd.get_dummies(data)

In [0]:
# 項目を削除
#model_data = model_data.drop(['del_columns1', 'del_columns2'], axis=1)



In [0]:
# データを確認
model_data.head(10)

In [0]:
X = model_data.drop(['y_no', 'y_yes'], axis=1)
y = model_data[['y_no', 'y_yes']]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [0]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42)

In [0]:
model.fit(X_train, y_train)

In [0]:
model.score(X_train, y_train)

In [0]:
model.score(X_test, y_test)

In [0]:
# モデル構築の中でどの変数が重要な役割を担ったのかを定量的に把握する
s = pd.Series(model.feature_importances_, 
              index=X_test.columns )

# 取得した値を降順に表示
s[:10].sort_values(ascending=False).plot.bar(width=0.8,color='C0')